In [1]:
import pygame
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from collections import deque


SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
FPS = 30
ENEMY_SIZE = 20
TOWER_SIZE = 40
SPAWN_RATE = 100
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLACK = (0, 0, 0)


class Tower:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.range = 250
        self.damage = 1
        self.reload_time = 60
        self.cooldown = 0

    def draw(self):
        pygame.draw.rect(screen, GREEN, (self.x, self.y, TOWER_SIZE, TOWER_SIZE))

    def in_range(self, enemy):
        dx = (self.x + TOWER_SIZE / 2) - (enemy.x + ENEMY_SIZE / 2)
        dy = (self.y + TOWER_SIZE / 2) - (enemy.y + ENEMY_SIZE / 2)
        return dx**2 + dy**2 <= self.range**2

    def attack(self, enemies):
        if self.cooldown == 0:
            hit = False
            for enemy in enemies:
                if self.in_range(enemy):
                    enemy.health -= self.damage
                    hit = True
            if hit:
                self.cooldown = self.reload_time
        else:
            self.cooldown -= 1


class Enemy:
    def __init__(self):
        self.x = 0
        self.y = random.randint(50, SCREEN_HEIGHT - 50 - ENEMY_SIZE)
        self.health = 5
        self.speed = 2

    def move(self):
        self.x += self.speed

    def draw(self):
        if self.health > 0:
            pygame.draw.rect(screen, RED, (self.x, self.y, ENEMY_SIZE, ENEMY_SIZE))

    def is_alive(self):
        return self.x < SCREEN_WIDTH and self.health > 0


def create_ppo_model():
    inputs = layers.Input(shape=(4,))
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(128, activation='relu')(x)
    action_probs = layers.Dense(2, activation='softmax')(x)
    value = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=[action_probs, value])
    return model

model = create_ppo_model()
optimizer = optimizers.Adam(learning_rate=0.01)

def get_action(state):
    state = tf.convert_to_tensor([state], dtype=tf.float32)
    action_probs, _ = model(state)
    return np.random.choice(2, p=np.squeeze(action_probs.numpy()))


pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()


enemies = []
towers = []
score = 0
frame_count = 0

running = True
while running:
    screen.fill(BLACK)
    if frame_count % SPAWN_RATE == 0:
        enemies.append(Enemy())

    for enemy in enemies[:]:
        enemy.move()
        enemy.draw()
        if not enemy.is_alive():
            enemies.remove(enemy)
            score += 1

    if frame_count % 300 == 0:
        action = get_action(np.random.rand(4))
        if action == 1:
            towers.append(Tower(400, random.randint(50, SCREEN_HEIGHT - 50 - TOWER_SIZE)))

    for tower in towers:
        tower.draw()
        tower.attack(enemies)

    pygame.display.flip()
    clock.tick(FPS)
    frame_count += 1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()

pygame 2.5.2 (SDL 2.28.3, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
